In [17]:
#import packages and formatting statements

import numpy as np
import math
# !pip install matplotlib
from matplotlib import pyplot as plt
# !pip install astropy==5.3
import astropy.io.fits as pyfits
from astropy.table import Table, join, Column
from astropy.wcs import WCS
from astropy.io import ascii
import os
from astropy.coordinates import SkyCoord, Angle
from astropy.nddata import Cutout2D
import astropy.units as u
from astropy.visualization.wcsaxes import SphericalCircle
from matplotlib.patches import Circle
from reproject import reproject_interp

import sys
sys.path.append('./Python')
from CheckInImage import check_in_image
from AngularSize import findAngSize

plt.rc('text',usetex=False)
fontsize=10
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

ext="HA_FLUX_CORR"

In [18]:
# Compile galaxy and sne data
data = Table.read('../Data/1.GroupedSampleData.csv', format='csv')

data[0:1]

col0,galaxy,distance,gal_ra,gal_dec,pos_ang,incl,nat_res,r_eff,maps_musedap,maps_nat,maps_150,maps_1kpc,maps_HII,maps_irac,maps_CO21,maps_eCO21,maps_CO_fine,maps_eCO_fine,maps_ext,maps_vdisp,maps_HST,JWST200,JWST300,JWST335,JWST360,JWST770,JWST1000,JWST1130,JWST2100,sn_name,sn_type,sn_ra,sn_dec
int64,str7,float64,float64,float64,float64,float64,float64,float64,str71,str58,str64,str63,str50,str40,str72,str73,str72,str73,str36,str32,str41,str101,str101,str101,str101,str101,str102,str102,str102,str82,str50,str127,str132
0,NGC1087,15.85,41.60492,-0.498717,359.1,42.9,0.92,42.091877,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_1kpc.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,../Data/IRAC/ngc1087_irac1_atGauss4.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_120pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_120pc_broad_emom0.fits,../Data/SFR_v1/NGC1087_MAPS_sfr.fits,../Data/vdisp/ngc1087_vdisp.fits,../../GalaxyFiles/HST/ngc1087_hst_ha.fits,../../GalaxyFiles/JWSTLargeFiles/ngc1087_nircam_lv3_f200w_i2d_anchor_atgauss0p85.fits,../../GalaxyFiles/JWSTLargeFiles/ngc1087_nircam_lv3_f300m_i2d_anchor_atgauss0p85.fits,../../GalaxyFiles/JWSTLargeFiles/ngc1087_nircam_lv3_f335m_i2d_anchor_atgauss0p85.fits,../../GalaxyFiles/JWSTLargeFiles/ngc1087_nircam_lv3_f360m_i2d_anchor_atgauss0p85.fits,../../GalaxyFiles/JWSTLargeFiles/ngc1087_miri_lv3_f770w_i2d_anchor_atgauss0p85.fits,../../GalaxyFiles/JWSTLargeFiles/ngc1087_miri_lv3_f1000w_i2d_anchor_atgauss0p85.fits,../../GalaxyFiles/JWSTLargeFiles/ngc1087_miri_lv3_f1130w_i2d_anchor_atgauss0p85.fits,../../GalaxyFiles/JWSTLargeFiles/ngc1087_miri_lv3_f2100w_i2d_anchor_atgauss0p85.fits,['SN1995V'],['II'],[41.61152777777777],[-0.4987861111111111]


In [19]:
    
# get plot information for main image maps for the zooms
def getPlotStuff(gal, img, HSTimg, HIIimg, CO21Img, eCO21Img, JWST200,JWST770,JWST2100, names, types, ras, decs, dists, ext):
    if os.path.isfile(img) and os.path.isfile(HIIimg):

        #HA map
        ha_hdu = pyfits.open(img)
        ha_map = ha_hdu[ext].data
        wcs_HA = WCS(ha_hdu[ext].header, naxis=2)

        #HST map
        hst_hdu = pyfits.open(HSTimg)
        hst_map = hst_hdu[0].data
        wcs_HST = WCS(hst_hdu[0].header, naxis=2)
        
        #CO + Sigma map
        co_hdu       = pyfits.open(CO21Img)
        eco_hdu      = pyfits.open(eCO21Img)
        co_map = co_hdu[0].data
        eco_map = eco_hdu[0].data
        sig_map = co_map/np.clip(eco_map,0.01,np.max(eco_map))
        wcs_co = WCS(co_hdu[0].header, naxis=2)
        
        #JWST200 map
        jwst200_hdu  = pyfits.open(JWST200)
        jwst200_map = jwst200_hdu[1].data
        wcs_jwst200 = WCS(jwst200_hdu[1].header, naxis=2)
        
        #JWST200 map
        jwst770_hdu  = pyfits.open(JWST770)
        jwst770_map = jwst770_hdu[1].data
        wcs_jwst770 = WCS(jwst770_hdu[1].header, naxis=2)
         
        #JWST2100 map
        jwst2100_hdu  = pyfits.open(JWST2100)
        jwst2100_map = jwst2100_hdu[1].data
        wcs_jwst2100 = WCS(jwst2100_hdu[1].header, naxis=2)                                
               
        #HII contours
        hii_hdu = pyfits.open(HIIimg)
        #MUSE
        hii_map = hii_hdu[0].data
        #HST   
        hii_map_HST, footprint = reproject_interp(hii_hdu, hst_hdu[0].header)
        #ALMA
        hii_map_CO, footprint = reproject_interp(hii_hdu, co_hdu[0].header)                       
        #JWST200
        hii_map_jwst200, footprint = reproject_interp(hii_hdu, jwst200_hdu[1].header)                                 
        #JWST770
        hii_map_jwst770, footprint = reproject_interp(hii_hdu, jwst770_hdu[1].header)                                 
        #JWST2100
        hii_map_jwst2100, footprint = reproject_interp(hii_hdu, jwst2100_hdu[1].header)
 
        #CO contours
        #MUSE
        co_map_MUSE,f  = reproject_interp(co_hdu, ha_hdu[ext].header)
        eco_map_MUSE,f = reproject_interp(eco_hdu, ha_hdu[ext].header)
        sig_map_MUSE = co_map_MUSE/np.clip(eco_map_MUSE,0.01,np.max(eco_map_MUSE))
        #HST   
        co_map_HST,f  = reproject_interp(co_hdu, hst_hdu[0].header)
        eco_map_HST,f = reproject_interp(eco_hdu, hst_hdu[0].header)
        sig_map_HST = co_map_HST/np.clip(eco_map_HST,0.01,np.max(eco_map_HST))
        #ALMA
        co_map_CO,f  = reproject_interp(co_hdu, co_hdu[0].header)
        eco_map_CO,f = reproject_interp(eco_hdu, co_hdu[0].header)
        sig_map_CO = co_map_CO/np.clip(eco_map_CO,0.01,np.max(eco_map_CO))
        #JWST200
        co_map_jwst200,f  = reproject_interp(co_hdu, jwst200_hdu[1].header)
        eco_map_jwst200,f = reproject_interp(eco_hdu, jwst200_hdu[1].header)
        sig_map_jwst200 = co_map_jwst200/np.clip(eco_map_jwst200,0.01,np.max(eco_map_jwst200))
        #JWST770
        co_map_jwst770,f  = reproject_interp(co_hdu, jwst770_hdu[1].header)
        eco_map_jwst770,f = reproject_interp(eco_hdu, jwst770_hdu[1].header)
        sig_map_jwst770 = co_map_jwst770/np.clip(eco_map_jwst770,0.01,np.max(eco_map_jwst770))
        #JWST2100
        co_map_jwst2100,f  = reproject_interp(co_hdu, jwst2100_hdu[1].header)
        eco_map_jwst2100,f = reproject_interp(eco_hdu, jwst2100_hdu[1].header)
        sig_map_jwst2100 = co_map_jwst2100/np.clip(eco_map_jwst2100,0.01,np.max(eco_map_jwst2100))                                 

    return(ha_map, wcs_HA, hst_map, wcs_HST, co_map, eco_map, sig_map_CO, wcs_co, jwst200_map, wcs_jwst200, jwst770_map, wcs_jwst770, jwst2100_map, wcs_jwst2100,  hii_map, hii_map_HST, hii_map_CO,hii_map_jwst200,hii_map_jwst770,hii_map_jwst2100, sig_map_MUSE,sig_map_HST,sig_map_jwst200,sig_map_jwst770,sig_map_jwst2100,dists)


In [29]:
# # set up plot information for MUSE zooms
# # set up plots, wcs information, and HII maps for contours.
plt.clf()
plt.rcParams['hatch.color'] = 'greenyellow'
current_cmap = plt.get_cmap("magma")
current_cmap.set_bad("black")


# supposed to be SNe but I'm iterating through galaxies
comp = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]
skipped = [3,5,7,8,9]
finished = [0,1,2,4,6]
for j in range(len(comp)):
    i = comp[j]
    galaxy=data["galaxy"][i]
#     if len(data["sn_name"][i] > 1):
        
    sn_ra   = [float(x.strip(' []')) for x in data["sn_ra"][i].split(',')]
    sn_dec  = [float(x.strip(' []')) for x in data["sn_dec"][i].split(',')]
    sn_name = [x.strip(' []') for x in data["sn_name"][i].split(',')]
    sn_type = [x.strip(' []') for x in data["sn_type"][i].split(',')]

    for j in range(len(sn_ra)):
        ha_map, wcs_HA, hst_map, wcs_HST, co_map, eco_map, sig_map, wcs_co, jwst200_map, wcs_jwst200, jwst770_map, wcs_jwst770, jwst2100_map, wcs_jwst2100,  hii_map, hii_map_HST, hii_map_CO,hii_map_jwst200,hii_map_jwst770,hii_map_jwst2100, sig_map_MUSE,sig_map_HST,sig_map_jwst200,sig_map_jwst770,sig_map_jwst2100,dists = getPlotStuff(data["galaxy"][i], data["maps_nat"][i], data["maps_HST"][i], data["maps_HII"][i], data["maps_CO_fine"][i], data["maps_eCO_fine"][i], data["JWST200"][i],data["JWST770"][i],data["JWST2100"][i], sn_name[j], sn_type[j], sn_ra[j], sn_dec[j], data["distance"][i], ext=ext)

        fig, axs = plt.subplots(3,2,figsize=(7,12))

        #set up data to use astropy's cutout2D to plot subsections of galaxy map
        logData = np.log10(ha_map)

        halfBox = findAngSize(500,dists)/2. # 500 pc in decimal degrees/2?
        #cutout2D needs skycoord position to carry units
        ra, dec = Angle(sn_ra[j] * u.degree), Angle(sn_dec[j] * u.degree)
        raRad, decRad  = ra.radian * u.rad, dec.radian * u.rad    
        position = SkyCoord(raRad, decRad) #position is center, use ra & dec of SN location
        size = u.Quantity((findAngSize(500,dists),findAngSize(500,dists)), u.degree) #size is size of box in arcsec 
#         size = size*6
        # make 2D cutout, will assign a new wcs to cutout to keep track of coords
        # use this new wcs when converting to pixels to add additional details to plot
        
        #MUSE maps
        cutout       = Cutout2D(logData, position, size, wcs_HA) 
        xval, yval   = cutout.wcs.wcs_world2pix(ra,dec, 0)
        HIICutout    = Cutout2D(hii_map, position, size, wcs_HA)
        sigCutout_HA = Cutout2D(sig_map_MUSE, position, size, wcs_HA)
        
        #HST maps
        HSTCutout = Cutout2D(hst_map, position, size, wcs_HST)
        HIICutoutHST = Cutout2D(hii_map_HST, position, size, wcs_HST)
        xvalHST, yvalHST = HSTCutout.wcs.wcs_world2pix(ra,dec, 0)      
        sigCutout_HST = Cutout2D(sig_map_HST, position, size, wcs_HST)        
        
        #ALMA maps
        coCutout  = Cutout2D(co_map, position, size, wcs_co)   
        sigCutout = Cutout2D(sig_map, position, size, wcs_co)
        xvalCO, yvalCO = coCutout.wcs.wcs_world2pix(ra,dec, 0)
        HIICutoutCO = Cutout2D(hii_map_CO, position, size, wcs_co)

        #JWST200 maps
        JWST200Cutout = Cutout2D(jwst200_map, position, size, wcs_jwst200)
        xvalJWST200, yvalJWST200 = JWST200Cutout.wcs.wcs_world2pix(ra,dec, 0)
        sigCutout_jwst200 = Cutout2D(sig_map_jwst200, position, size, wcs_jwst200)
        HIICutout_jwst200 = Cutout2D(hii_map_jwst200, position, size, wcs_jwst200)

        #JWST770 maps
        JWST770Cutout = Cutout2D(jwst770_map, position, size, wcs_jwst770)
        xvalJWST770, yvalJWST770 = JWST770Cutout.wcs.wcs_world2pix(ra,dec, 0)
        sigCutout_jwst770 = Cutout2D(sig_map_jwst770, position, size, wcs_jwst770)
        HIICutout_jwst770 = Cutout2D(hii_map_jwst770, position, size, wcs_jwst770)
                
        #JWST2100 maps
        JWST2100Cutout = Cutout2D(jwst2100_map, position, size, wcs_jwst2100)
        xvalJWST2100, yvalJWST2100 = JWST2100Cutout.wcs.wcs_world2pix(ra,dec, 0)
        sigCutout_jwst2100 = Cutout2D(sig_map_jwst2100, position, size, wcs_jwst2100)
        HIICutout_jwst2100 = Cutout2D(hii_map_jwst2100, position, size, wcs_jwst2100)
        
        titleStr = sn_name[j].upper().strip("'") + " (" + galaxy.upper() + ") Type " + sn_type[j].strip("'")

        # make plot
        fig.suptitle(titleStr, fontsize=18) 
        #MUSE
        axs[0][0].set_title("VLT/MUSE", fontsize=14)
        axs[0][0].imshow(cutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(cutout.data, 5), vmax=np.percentile(cutout.data, 95),interpolation = "nearest", zorder = 0)
        axs[0][0].scatter(xval, yval, color = "black", marker ="o", s = 60, zorder=2)
        axs[0][0].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2)
        axs[0][0].set_ylabel(" ")
        axs[0][0].set_xlabel(" ")
        axs[0][0].set_xticklabels([])
        axs[0][0].set_yticklabels([])
        axs[0][0].tick_params(axis = "both", direction = "in", length = 0)
        axs[0][0].contourf(HIICutout.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
        axs[0][0].contour(HIICutout.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
        axs[0][0].contourf(sigCutout_HA.data, [3,1000000], colors = ["white"], linestyles = ("-"), zorder = 1, alpha=0.1)
        axs[0][0].contour(sigCutout_HA.data, [3], colors = ["white"], linestyles = ("-"), linewidths=2, zorder = 1)
        #HST
        axs[0][1].set_title("HST", fontsize=14)
        axs[0][1].imshow(HSTCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(HSTCutout.data, 5), vmax=np.percentile(HSTCutout.data,95),interpolation = "nearest", zorder = 0)
        axs[0][1].scatter(xvalHST, yvalHST, color = "black", marker ="o", s = 60, zorder=2)
        axs[0][1].scatter(xvalHST, yvalHST, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2)
        axs[0][1].set_ylabel(" ")
        axs[0][1].set_xlabel(" ")
        axs[0][1].set_xticklabels([])
        axs[0][1].set_yticklabels([])
        axs[0][1].tick_params(axis = "both", direction = "in", length = 0)
        axs[0][1].contourf(HIICutoutHST.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
        axs[0][1].contour(HIICutoutHST.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
        axs[0][1].contourf(sigCutout_HST.data, [3,1000000], colors = ["white"], linestyles = ("-"), zorder = 1, alpha=0.1)
        axs[0][1].contour(sigCutout_HST.data, [3], colors = ["white"], linestyles = ("-"), linewidths=2, zorder = 1)
        #ALMA
        axs[1][0].set_title("ALMA", fontsize=14)
        axs[1][0].imshow(coCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(coCutout.data, 5), vmax=np.percentile(coCutout.data, 95),interpolation = "nearest", zorder = 0)
        axs[1][0].scatter(xvalCO, yvalCO, color = "black", marker ="o", s = 60, zorder=2)
        axs[1][0].scatter(xvalCO, yvalCO, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2)
        axs[1][0].set_ylabel(" ")
        axs[1][0].set_xlabel(" ")
        axs[1][0].set_xticklabels([])
        axs[1][0].set_yticklabels([])
        axs[1][0].tick_params(axis = "both", direction = "in", length = 0)
        axs[1][0].contourf(HIICutoutCO.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
        axs[1][0].contour(HIICutoutCO.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
        axs[1][0].contourf(sigCutout.data, [3,1000000], colors = ["white"], linestyles = ("-"), zorder = 1, alpha=0.1)
        axs[1][0].contour(sigCutout.data, [3], colors = ["white"], linestyles = ("-"), linewidths=2, zorder = 1)
        #JWST200
        axs[1][1].set_title("JWST NIRCam F200W", fontsize=14)
        axs[1][1].imshow(np.log10(JWST200Cutout.data), cmap=current_cmap, aspect="equal", origin = "lower",interpolation = "nearest", zorder = 0)
        axs[1][1].scatter(xvalJWST200, yvalJWST200, color = "black", marker ="o", s = 60, zorder=2)
        axs[1][1].scatter(xvalJWST200, yvalJWST200, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2)
        axs[1][1].set_ylabel(" ")
        axs[1][1].set_xlabel(" ")
        axs[1][1].set_xticklabels([])
        axs[1][1].set_yticklabels([])
        axs[1][1].tick_params(axis = "both", direction = "in", length = 0)
        axs[1][1].contourf(HIICutout_jwst200.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
        axs[1][1].contour(HIICutout_jwst200.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
        axs[1][1].contourf(sigCutout_jwst200.data, [3,1000000], colors = ["white"], linestyles = ("-"), zorder = 1, alpha=0.1)
        axs[1][1].contour(sigCutout_jwst200.data, [3], colors = ["white"], linestyles = ("-"), linewidths=2, zorder = 1)
        #JWST770
        axs[2][0].set_title("JWST MIRI F770W", fontsize=14)
        axs[2][0].imshow(np.log10(JWST770Cutout.data), cmap=current_cmap, aspect="equal", origin = "lower",interpolation = "nearest", zorder = 0)
        axs[2][0].scatter(xvalJWST770, yvalJWST770, color = "black", marker ="o", s = 60, zorder=2)
        axs[2][0].scatter(xvalJWST770, yvalJWST770, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2)
        axs[2][0].set_ylabel(" ")
        axs[2][0].set_xlabel(" ")
        axs[2][0].set_xticklabels([])
        axs[2][0].set_yticklabels([])
        axs[2][0].tick_params(axis = "both", direction = "in", length = 0)
        axs[2][0].contourf(HIICutout_jwst770.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
        axs[2][0].contour(HIICutout_jwst770.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
        axs[2][0].contourf(sigCutout_jwst770.data, [3,1000000], colors = ["white"], linestyles = ("-"),  zorder = 1, alpha=0.1)
        axs[2][0].contour(sigCutout_jwst770.data, [3], colors = ["white"], linestyles = ("-"), linewidths=2, zorder = 1)
        #JWST2100
        axs[2][1].set_title("JWST MIRI F2100W", fontsize=14)
        axs[2][1].imshow(np.log10(JWST2100Cutout.data), cmap=current_cmap, aspect="equal", origin = "lower",interpolation = "nearest", zorder = 0)
        axs[2][1].scatter(xvalJWST2100, yvalJWST2100, color = "black", marker ="o", s = 60, zorder=2)
        axs[2][1].scatter(xvalJWST2100, yvalJWST2100, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2)
        axs[2][1].set_ylabel(" ")
        axs[2][1].set_xlabel(" ")
        axs[2][1].set_xticklabels([])
        axs[2][1].set_yticklabels([])
        axs[2][1].tick_params(axis = "both", direction = "in", length = 0)
        axs[2][1].contourf(HIICutout_jwst2100.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
        axs[2][1].contour(HIICutout_jwst2100.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
        axs[2][1].contourf(sigCutout_jwst2100.data, [3,1000000], colors = ["white"], linestyles = ("-"), zorder = 1, alpha=0.1)
        axs[2][1].contour(sigCutout_jwst2100.data, [3], colors = ["white"], linestyles = ("-"), linewidths=2, zorder = 1)

        plt.tight_layout(pad=0.4, w_pad=0, h_pad=0.2)
        plt.subplots_adjust(top=0.95)
        plt.show()
        figname = "../Figures/SixPanel_" + galaxy.upper() +"_" + sn_name[j] + ".png"
        fig.savefig(figname, dpi=300)
        plt.close()
        plt.clf()


IndexError: index 10 is out of bounds for axis 0 with size 10

<Figure size 640x480 with 0 Axes>

In [ ]:
for i in range(len(data["galaxy"])):
    print(data[sn_name][i])

resPC = [71,82,109,109,109,109,82,69,69,69,90,90,58,58,58,58,58,57,57,57,57,64,64,64,64,64,64,64,85,85,85,85,85]
len(resPC)
print(data["galaxy"][4])

In [ ]:
# plt.rcParams['hatch.color'] = 'greenyellow'
# numPixels, numHIIPixels = [],[]

# current_cmap = plt.get_cmap("magma")
# current_cmap.set_bad("black")

# NUM_SNE= 33

# fig, axs = plt.subplots(2,2,figsize=(7,9))

# # for j in range(len(sn_gals)): # counter to count galaxy maps
# for j in range(0,1):
    
#     #set up legend label with type classification
#     if sn_types[j] != "Unclassified":
#         leglab = "Type " + sn_types[j].strip("'")
#     else:
#         leglab= sn_types[j]
#     legStr = str(resPC[j]) + " pc"
    
#     #set up data to use astropy's cutout2D to plot subsections of galaxy map
#     data = plots[j]
#     logData = np.log10(data)
#     HIIData = HIImaps[j]
#     HSTdata = HSTmaps[j]
# #     logHST = np.log10(HSTdata)
#     sigData = sigma[j]
#     coData = co_maps[j]
#     ecoData = eco_maps[j]
#     HIIhstData = HIIhst[j]

#     wcs=wcss[j]

#     halfBox = findAngSize(500,dists[j])/2. # 500 pc in decimal degrees/2?
#     #cutout2D needs skycoord position to carry units
#     ra, dec = Angle(sn_ras[j] * u.degree), Angle(sn_decs[j] * u.degree)
#     raRad, decRad  = ra.radian * u.rad, dec.radian * u.rad    
#     position = SkyCoord(raRad, decRad) #position is center, use ra & dec of SN location
#     size = u.Quantity((findAngSize(500,dists[j]),findAngSize(500,dists[j])), u.degree) #size is size of box in arcsec 
#     # make 2D cutout, will assign a new wcs to cutout to keep track of coords
#     cutout = Cutout2D(logData, position, size, wcs) 
#     # use this new wcs when converting to pixels to add additional details to plot
#     xval, yval = cutout.wcs.wcs_world2pix(ra,dec, 0)
#     # get HII data for contours
#     HIICutout = Cutout2D(HIIData, position, size, wcs)
#     HIICutoutHST = Cutout2D(HIIhstData, position, size, wcsHST[j])
#     sigCutout = Cutout2D(sigData, position, size, wcs)
#     coCutout  = Cutout2D(coData, position, size, wcs)    
#     ecoCutout = Cutout2D(ecoData, position, size, wcs)    
#     HSTCutout = Cutout2D(HSTdata, position, size, wcsHST[j])
#     xvalHST, yvalHST = HSTCutout.wcs.wcs_world2pix(ra,dec, 0)

#     titleStr = sn_names[j].upper().strip("'") + " (" + sn_gals[j].upper() + ") Type " + sn_types[j].strip("'")
            
#     # make plot
#     fig.suptitle(titleStr, fontsize=18) 
#     axs[0][0].set_title("MUSE", fontsize=14)
#     axs[0][0].imshow(cutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(cutout.data, 5), vmax=np.percentile(cutout.data, 95),interpolation = "nearest", zorder = 0)
#     axs[0][0].scatter(xval, yval, color = "black", marker ="o", s = 60, zorder=2)
#     axs[0][0].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#     axs[0][0].set_ylabel(" ")
#     axs[0][0].set_xlabel(" ")
#     axs[0][0].set_xticklabels([])
#     axs[0][0].set_yticklabels([])
#     axs[0][0].tick_params(axis = "both", direction = "in", length = 0)
#     axs[0][1].set_title("HST", fontsize=14)
#     axs[0][1].imshow(HSTCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(HSTCutout.data, 5), vmax=np.percentile(HSTCutout.data,95),interpolation = "nearest", zorder = 0)
#     axs[0][1].scatter(xvalHST, yvalHST, color = "black", marker ="o", s = 60, zorder=2)
#     axs[0][1].scatter(xvalHST, yvalHST, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#     axs[0][1].set_ylabel(" ")
#     axs[0][1].set_xlabel(" ")
#     axs[0][1].set_xticklabels([])
#     axs[0][1].set_yticklabels([])
#     axs[0][1].tick_params(axis = "both", direction = "in", length = 0)
    

#     axs[0][0].contourf(HIICutout.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
#     axs[0][0].contour(HIICutout.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
#     axs[0][1].contourf(HIICutoutHST.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
#     axs[0][1].contour(HIICutoutHST.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)

    
#     axs[1][0].set_title("CO (2-1)", fontsize=14)
#     axs[1][0].imshow(coCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(coCutout.data, 5), vmax=np.percentile(coCutout.data, 95),interpolation = "nearest", zorder = 0)
#     axs[1][0].scatter(xval, yval, color = "black", marker ="o", s = 60, zorder=2)
#     axs[1][0].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#     axs[1][0].set_ylabel(" ")
#     axs[1][0].set_xlabel(" ")
#     axs[1][0].set_xticklabels([])
#     axs[1][0].set_yticklabels([])
#     axs[1][0].tick_params(axis = "both", direction = "in", length = 0)
#     axs[1][1].set_title("S/N CO (2-1)", fontsize=14)
#     axs[1][1].imshow(sigCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(sigCutout.data, 5), vmax=np.percentile(sigCutout.data,95),interpolation = "nearest", zorder = 0)
#     axs[1][1].scatter(xval, yval, color = "black", marker ="o", s = 60, zorder=2)
#     axs[1][1].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#     axs[1][1].set_ylabel(" ")
#     axs[1][1].set_xlabel(" ")
#     axs[1][1].set_xticklabels([])
#     axs[1][1].set_yticklabels([])
#     axs[1][1].tick_params(axis = "both", direction = "in", length = 0)
    

#     axs[1][0].contourf(sigCutout.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
#     axs[1][0].contour(sigCutout.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
    
# #     axs[1][1].contourf(HIICutoutHST.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
# #     axs[1][1].contour(HIICutoutHST.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)

    
    
    
# plt.tight_layout(pad=0.4, w_pad=0, h_pad=0.2)
# plt.show()
# fig.savefig("../Figures/HST_MUSE_CO_Comp.png", dpi=300)
# plt.close()

In [ ]:
# fig = plt.figure(constrained_layout=True, figsize=(10,200))

# i=0

# comp = [0,1,4,6,7,8,9,10,11,13,14,15,20,28,29,31,32]


# # create 3x1 subfigs
# subfigs = fig.subfigures(nrows=len(comp), ncols=1)
# print(subfigs)

# for row, subfig in enumerate(subfigs):
#     titleStr = sn_names[comp[i]].upper().strip("'") + " (" + sn_gals[comp[i]].upper() + ") Type " + sn_types[comp[i]].strip("'")
#     subfig.suptitle(titleStr, fontsize=18)
    
#     #set up data to use astropy's cutout2D to plot subsections of galaxy map
#     data = plots[comp[i]]
#     logData = np.log10(data)
#     HIIData = HIImaps[comp[i]]
#     HSTdata = HSTmaps[comp[i]]
#     HIIhstData = HIIhst[comp[i]]
#     sigData = sigma[comp[i]]
#     coData = co_maps[comp[i]]
#     ecoData = eco_maps[comp[i]]

#     wcs=wcss[comp[i]]
    
#     halfBox = findAngSize(500,dists[comp[i]])/2. # 500 pc in decimal degrees/2?
#     #cutout2D needs skycoord position to carry units
#     ra, dec = Angle(sn_ras[comp[i]] * u.degree), Angle(sn_decs[comp[i]] * u.degree)
#     raRad, decRad  = ra.radian * u.rad, dec.radian * u.rad    
#     position = SkyCoord(raRad, decRad) #position is center, use ra & dec of SN location
#     size = u.Quantity((findAngSize(500,dists[comp[i]]),findAngSize(500,dists[comp[i]])), u.degree) #size is size of box in arcsec 
#     # make 2D cutout, will assign a new wcs to cutout to keep track of coords
#     cutout = Cutout2D(logData, position, size, wcs) 
#     # use this new wcs when converting to pixels to add additional details to plot
#     xval, yval = cutout.wcs.wcs_world2pix(ra,dec, 0)
#     # get HII data for contours
#     HIICutout = Cutout2D(HIIData, position, size, wcs)
#     HIICutoutHST = Cutout2D(HIIhstData, position, size, wcsHST[comp[i]])
#     HSTCutout = Cutout2D(HSTdata, position, size, wcsHST[comp[i]])
#     xvalHST, yvalHST = HSTCutout.wcs.wcs_world2pix(ra,dec, 0)
#     sigCutout = Cutout2D(sigData, position, size, wcs)
#     coCutout  = Cutout2D(coData, position, size, wcs)    
#     ecoCutout = Cutout2D(ecoData, position, size, wcs) 

#     # create 1x3 subplots per subfig
#     axs = subfig.subplots(nrows=2, ncols=2)
#     for row, col in enumerate(axs):
#         axs[0][0].set_title("MUSE", fontsize=14)
#         axs[0][0].imshow(cutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(cutout.data, 5), vmax=np.percentile(cutout.data, 95),interpolation = "nearest", zorder = 0)
#         axs[0][0].scatter(xval, yval, color = "black", marker ="o", s = 60, zorder=2)
#         axs[0][0].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#         axs[0][0].set_ylabel(" ")
#         axs[0][0].set_xlabel(" ")
#         axs[0][0].set_xticklabels([])
#         axs[0][0].set_yticklabels([])
#         axs[0][0].tick_params(axis = "both", direction = "in", length = 0)
#         axs[0][1].set_title("HST", fontsize=14)
#         axs[0][1].imshow(HSTCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(HSTCutout.data, 5), vmax=np.percentile(HSTCutout.data,95),interpolation = "nearest", zorder = 0)
#         axs[0][1].scatter(xvalHST, yvalHST, color = "black", marker ="o", s = 60, zorder=2)
#         axs[0][1].scatter(xvalHST, yvalHST, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#         axs[0][1].set_ylabel(" ")
#         axs[0][1].set_xlabel(" ")
#         axs[0][1].set_xticklabels([])
#         axs[0][1].set_yticklabels([])
#         axs[0][1].tick_params(axis = "both", direction = "in", length = 0)


#         axs[0][0].contourf(HIICutout.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
#         axs[0][0].contour(HIICutout.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)
#         axs[0][1].contourf(HIICutoutHST.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
#         axs[0][1].contour(HIICutoutHST.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)


#         axs[1][0].set_title("CO (2-1)", fontsize=14)
#         axs[1][0].imshow(coCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(coCutout.data, 5), vmax=np.percentile(coCutout.data, 95),interpolation = "nearest", zorder = 0)
#         axs[1][0].scatter(xval, yval, color = "black", marker ="o", s = 60, zorder=2)
#         axs[1][0].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#         axs[1][0].set_ylabel(" ")
#         axs[1][0].set_xlabel(" ")
#         axs[1][0].set_xticklabels([])
#         axs[1][0].set_yticklabels([])
#         axs[1][0].tick_params(axis = "both", direction = "in", length = 0)
#         axs[1][1].set_title("S/N CO (2-1)", fontsize=14)
#         axs[1][1].imshow(sigCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(sigCutout.data, 5), vmax=np.percentile(sigCutout.data,95),interpolation = "nearest", zorder = 0)
#         axs[1][1].scatter(xval, yval, color = "black", marker ="o", s = 60, zorder=2)
#         axs[1][1].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#         axs[1][1].set_ylabel(" ")
#         axs[1][1].set_xlabel(" ")
#         axs[1][1].set_xticklabels([])
#         axs[1][1].set_yticklabels([])
#         axs[1][1].tick_params(axis = "both", direction = "in", length = 0)


#         axs[1][0].contourf(sigCutout.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
#         axs[1][0].contour(sigCutout.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=2, zorder = 1)



#     i+=1
#     print(i)
# plt.tight_layout(pad=0.4, w_pad=0, h_pad=0.2)
# plt.show()
# fig.savefig("../Figures/MUSE_HST_CO_Zooms_500pc.png", dpi=300)
# plt.close()


In [ ]:
# fig = plt.figure(constrained_layout=True, figsize=(10,100))

# i=0

# comp = [0,1,4,6,7,8,9,10,11,13,14,15,20,28,29,31,32]


# # create 3x1 subfigs
# subfigs = fig.subfigures(nrows=len(comp), ncols=1)

# for row, subfig in enumerate(subfigs):
#     titleStr = sn_names[comp[i]].upper().strip("'") + " (" + sn_gals[comp[i]].upper() + ") Type " + sn_types[comp[i]].strip("'")
#     subfig.suptitle(titleStr, fontsize=18)
    
#     #set up data to use astropy's cutout2D to plot subsections of galaxy map
#     data = plots[comp[i]]
#     logData = np.log10(data)
#     HIIData = HIImaps[comp[i]]
#     HSTdata = HSTmaps[comp[i]]
#     HIIhstData = HIIhst[comp[i]]

#     wcs=wcss[comp[i]]

#     halfBox = findAngSize(500,dists[comp[i]])/2. # 500 pc in decimal degrees/2?
#     #cutout2D needs skycoord position to carry units
#     ra, dec = Angle(sn_ras[comp[i]] * u.degree), Angle(sn_decs[comp[i]] * u.degree)
#     raRad, decRad  = ra.radian * u.rad, dec.radian * u.rad    
#     position = SkyCoord(raRad, decRad) #position is center, use ra & dec of SN location
#     size = u.Quantity((findAngSize(500,dists[comp[i]]),findAngSize(500,dists[comp[i]])), u.degree) #size is size of box in arcsec 
#     # make 2D cutout, will assign a new wcs to cutout to keep track of coords
#     cutout = Cutout2D(logData, position, size, wcs) 
#     # use this new wcs when converting to pixels to add additional details to plot
#     xval, yval = cutout.wcs.wcs_world2pix(ra,dec, 0)
#     # get HII data for contours
#     HIICutout = Cutout2D(HIIData, position, size, wcs)
#     HIICutoutHST = Cutout2D(HIIhstData, position, size, wcsHST[comp[i]])
#     HSTCutout = Cutout2D(HSTdata, position, size, wcsHST[comp[i]])
#     xvalHST, yvalHST = HSTCutout.wcs.wcs_world2pix(ra,dec, 0)


#     # create 1x2 subplots per subfig
#     ax = subfig.subplots(nrows=1, ncols=2)
#     for col in ax:
#         ax[0].imshow(cutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(cutout.data, 5), vmax=np.percentile(cutout.data, 95),interpolation = "nearest", zorder = 0)
#         ax[0].scatter(xval, yval, color = "white", marker ="o", s = 60, zorder=2)
#         ax[0].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
#         ax[0].set_title("MUSE", fontsize=14) 
#         ax[0].set_ylabel(" ")
#         ax[0].set_xlabel(" ")
#         ax[0].set_xticklabels([])
#         ax[0].set_yticklabels([])
#         ax[0].tick_params(axis = "both", direction = "in", length = 0)
        
#         ax[1].imshow(HSTCutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=np.percentile(HSTCutout.data, 5), vmax=np.percentile(HSTCutout.data, 95),interpolation = "nearest", zorder = 0)
#         ax[1].scatter(xvalHST, yvalHST, color = "white", marker ="o", s = 60, zorder=2)
#         ax[1].scatter(xvalHST, yvalHST, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2)
#         ax[1].set_title("HST", fontsize=14)
#         ax[1].set_ylabel(" ")
#         ax[1].set_xlabel(" ")
#         ax[1].set_xticklabels([])
#         ax[1].set_yticklabels([])
#         ax[1].tick_params(axis = "both", direction = "in", length = 0)


#         ax[0].contourf(HIICutout.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
#         ax[0].contour(HIICutout.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=1, zorder = 1)
#         ax[1].contourf(HIICutoutHST.data, [3,1000000], colors = ["greenyellow"], linestyles = ("-"), hatches=['/'], zorder = 1, alpha=0.1)
#         ax[1].contour(HIICutoutHST.data, [3], colors = ["greenyellow"], linestyles = ("-"), linewidths=1, zorder = 1)
        
        
        
#     i+=1
# # plt.tight_layout(pad=0.5, w_pad=.8, h_pad=1.0)
# plt.show()
# fig.savefig("../Figures/MUSE_HST_Zooms_500pc.png", dpi=300)
# plt.close()